In [1]:
conda install -c conda-forge pdf2image

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install -c conda-forge poppler

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [25]:
#paths
source_file = '/tmp/sample_cards_file.pdf'
out_dir = '/tmp/process/'
 
# page setup
rows = 3
cols = 3
 
# defines where the last run of this left off on (first item will be 1.jpg if 0)
starting_count = 0
 
#spacing offsets
top_offest = 0
left_offset = 260
bottom_offset = 0
right_offset = 0
spacer = 40
vert_spacer=0

In [26]:
from pdf2image import convert_from_path
 
# function that converts multipage pdf to individual
# jpeg images. Function returns list of image paths.
def convert_pdf_to_jpegs(pdf_path, out_dir):
    file_paths=[]
    pages = convert_from_path(pdf_path, 500)
    page_count = 1
    for page in pages:
        image_path="{}temp_page_{}.jpg".format(out_dir, str(page_count))
        #add to the list
        file_paths.append(image_path)
        #save converted file
        page.save(image_path, 'JPEG')
        page_count += 1
    return file_paths

In [32]:
from PIL import Image

def split_images_from_page(image_path, out_dir, row_count, col_count, start_number):

    Im = Image.open(image_path)
    full_width = Im.width
    full_height = Im.height
    image_height = int((full_height - vert_spacer - top_offest - bottom_offset)/ rows)
    image_width = int((full_width - spacer - left_offset - right_offset)/ cols)
    
    
    
    image_count = 0
    row_current=1
    while row_current <= row_count:
        col_current = 1
        while col_current <= col_count:
            
            croppedIm = Im.crop((left_offset + ((col_current - 1) * image_width) + spacer, top_offest + ((row_current - 1) * image_height), min(left_offset + (col_current * image_width) + ((col_current - 1) * spacer), Im.width), top_offest + (row_current * image_height) + vert_spacer))
            #croppedIm = Im.crop((left_offset+((col_current-1)*image_width)+spacer, top_offest+((row_current-1)*image_height), min(left_offset+(col_current*image_width)+((col_current-1)*spacer), Im.width), top_offest+(row_current*image_height)+vert_spacer))
            croppedIm.save("{}/{}.jpg".format(out_dir, start_number+image_count))
       
            col_current+=1
            image_count+=1        
        row_current+=1

 

In [33]:
import os
 
page_count = 1

# convert pdf to jpeg file for each page
file_paths = convert_pdf_to_jpegs(source_file, out_dir)
 
# split each page into images
for file_path in file_paths:
    number_start = starting_count + (page_count-1) * (rows * cols) + 1
    split_images_from_page(file_path, out_dir, rows, cols, number_start)
    page_count += 1
 
# clean up delete jpeg files for each page
for file_path in file_paths:
    os.remove(file_path)